In [1]:
import os
os.environ['JSBSIM_ROOT_DIR'] = '/home/ubuntu/jsbsim-JSBSim-trusty-v2018a'

import sys
print ('\n'.join(sys.path))

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gym
import ray
from ray.rllib.agents.ppo import PPOAgent, DEFAULT_CONFIG
from ray.tune.logger import pretty_print

import boto3

import json


/home/ubuntu/src/cntk/bindings/python
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages
/home/ubuntu/gym-jsbsim
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions
/home/ubuntu/.ipython
altitude_sl_ft
heading_deg
pitch_rad
roll_rad
v_north_fps
v_east_fps
v_down_fps
aileron_cmd
elevator_cmd
rudder_cmd
throttle_cmd


In [2]:
# start up ray
ray.init(ignore_reinit_error=True)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-01-31_16-23-03_11276/logs.
Waiting for redis server at 127.0.0.1:51551 to respond...
Waiting for redis server at 127.0.0.1:61736 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 20.0 GB memory using /dev/shm.

View the web UI at http://localhost:8892/notebooks/ray_ui.ipynb?token=8833ee6f07a7fc6f1614be76ed260ab7682922d96d4ffe86



{'node_ip_address': None,
 'redis_address': '172.31.4.39:51551',
 'object_store_address': '/tmp/ray/session_2019-01-31_16-23-03_11276/sockets/plasma_store',
 'webui_url': 'http://localhost:8892/notebooks/ray_ui.ipynb?token=8833ee6f07a7fc6f1614be76ed260ab7682922d96d4ffe86',
 'raylet_socket_name': '/tmp/ray/session_2019-01-31_16-23-03_11276/sockets/raylet'}

In [19]:
config = DEFAULT_CONFIG.copy()
# num_workers is the number of actors that the agent will create. This determines the degree of parallelism that will be used.
config['num_workers'] = 5
#num_sgd_iter is the number of epochs of SGD (passes through the data) that will be used to optimize the PPO surrogate objective at each iteration of PPO
config['num_sgd_iter'] = 30
# sgd_minibatch_size is the SGD batch size that will be used to optimize the PPO surrogate objective.
config['sgd_minibatch_size'] = 512
config['model']['fcnet_hiddens'] = [100, 100]
config['num_cpus_per_worker'] = 0  # This avoids running out of resources in the notebook environment when this cell is re-executed
config["num_gpus"] = 1

agent = PPOAgent(config, 'JSBSim-HeadingControlTask-A320-NoFG-v0')

2019-01-31 17:01:25,704	WARNING ppo.py:137 -- By default, observations will be normalized with MeanStdFilter
2019-01-31 17:01:25,707	INFO policy_evaluator.py:260 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


state_variables =  (BoundedProperty(name='position/h-sl-ft', description='altitude above mean sea level [ft]', min=-1400, max=85000), BoundedProperty(name='attitude/psi-deg', description='heading [deg]', min=0, max=360), BoundedProperty(name='attitude/pitch-rad', description='pitch [rad]', min=-1.5707963267948966, max=1.5707963267948966), BoundedProperty(name='attitude/roll-rad', description='roll [rad]', min=-3.141592653589793, max=3.141592653589793), BoundedProperty(name='velocities/v-north-fps', description='velocity true north [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-east-fps', description='velocity east [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-down-fps', description='velocity downwards [ft/s]', min=-inf, max=inf))
action_variables =  (BoundedProperty(name='fcs/aileron-cmd-norm', description='aileron commanded position, normalised', min=-1.0, max=1.0), BoundedProperty(name='fcs/elevator-cmd-norm', description='elevator commanded positio

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-01-31 17:01:27,014	INFO multi_gpu_optimizer.py:74 -- LocalMultiGPUOptimizer devices ['/gpu:0']


In [30]:
for i in range(1000):
    print("start training ", i)
    result = agent.train()
    print(pretty_print(result))

start training  0


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ray/tune/logger.py:251: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(value, float):


custom_metrics: {}
date: 2019-01-31_17-23-29
done: false
episode_len_mean: 227.33
episode_reward_max: 711.7178964750779
episode_reward_mean: 499.4327954690493
episode_reward_min: 22.19121995665403
episodes_this_iter: 17
episodes_total: 4954
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.5062499642372131
    cur_lr: 4.999999873689376e-05
    entropy: 2.773599147796631
    kl: 0.007841161452233791
    policy_loss: -0.018025178462266922
    total_loss: 1430.773193359375
    vf_explained_var: 0.6375144124031067
    vf_loss: 1430.7872314453125
  grad_time_ms: 1106.266
  load_time_ms: 1.577
  num_steps_sampled: 1204000
  num_steps_trained: 1204000
  sample_time_ms: 2359.764
  update_time_ms: 7.92
iterations_since_restore: 301
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 1081.5755982398987
time_this_iter_s: 3.439323663711548
time_total_s: 1081.5755982398987
timestamp: 154

custom_metrics: {}
date: 2019-01-31_17-23-57
done: false
episode_len_mean: 219.94
episode_reward_max: 712.5394589356746
episode_reward_mean: 472.40117897850695
episode_reward_min: 42.11701695992685
episodes_this_iter: 18
episodes_total: 5104
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.5062499642372131
    cur_lr: 4.999999873689376e-05
    entropy: 2.9390196800231934
    kl: 0.012734102085232735
    policy_loss: -0.021088873967528343
    total_loss: 1817.0494384765625
    vf_explained_var: 0.578528881072998
    vf_loss: 1817.0643310546875
  grad_time_ms: 1114.025
  load_time_ms: 1.515
  num_steps_sampled: 1236000
  num_steps_trained: 1236000
  sample_time_ms: 2350.451
  update_time_ms: 7.963
iterations_since_restore: 309
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 1109.5052993297577
time_this_iter_s: 3.4823262691497803
time_total_s: 1109.5052993297577
timestamp:

custom_metrics: {}
date: 2019-01-31_17-24-25
done: false
episode_len_mean: 223.89
episode_reward_max: 713.1252637377797
episode_reward_mean: 488.51920756978876
episode_reward_min: -18.181044318977143
episodes_this_iter: 20
episodes_total: 5249
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.5062499642372131
    cur_lr: 4.999999873689376e-05
    entropy: 2.6274302005767822
    kl: 0.011454939842224121
    policy_loss: -0.011624453589320183
    total_loss: 1894.9342041015625
    vf_explained_var: 0.5371042490005493
    vf_loss: 1894.9400634765625
  grad_time_ms: 1116.262
  load_time_ms: 1.564
  num_steps_sampled: 1268000
  num_steps_trained: 1268000
  sample_time_ms: 2341.982
  update_time_ms: 7.665
iterations_since_restore: 317
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 1137.3257811069489
time_this_iter_s: 3.4804039001464844
time_total_s: 1137.3257811069489
timesta

...

custom_metrics: {}
date: 2019-01-31_18-19-53
done: false
episode_len_mean: 268.49
episode_reward_max: 768.101154012347
episode_reward_mean: 676.1120311201263
episode_reward_min: 53.246530527149226
episodes_this_iter: 15
episodes_total: 19056
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.7306471467018127
    cur_lr: 4.999999873689376e-05
    entropy: -0.011851872317492962
    kl: 0.00973135232925415
    policy_loss: -0.016933051869273186
    total_loss: 424.6548767089844
    vf_explained_var: 0.9022002220153809
    vf_loss: 424.66473388671875
  grad_time_ms: 1131.622
  load_time_ms: 1.574
  num_steps_sampled: 5044000
  num_steps_trained: 5044000
  sample_time_ms: 2351.446
  update_time_ms: 7.423
iterations_since_restore: 1261
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 4451.24392580986
time_this_iter_s: 3.4492945671081543
time_total_s: 4451.24392580986
timestamp: 

custom_metrics: {}
date: 2019-01-31_18-20-23
done: false
episode_len_mean: 257.83
episode_reward_max: 769.2629568411271
episode_reward_mean: 644.6084259427416
episode_reward_min: 227.08663049703267
episodes_this_iter: 16
episodes_total: 19180
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.7306471467018127
    cur_lr: 4.999999873689376e-05
    entropy: 0.13760001957416534
    kl: 0.0074261887930333614
    policy_loss: -0.021429074928164482
    total_loss: 1360.9412841796875
    vf_explained_var: 0.6942943930625916
    vf_loss: 1360.957275390625
  grad_time_ms: 1203.738
  load_time_ms: 1.651
  num_steps_sampled: 5076000
  num_steps_trained: 5076000
  sample_time_ms: 2430.651
  update_time_ms: 8.06
iterations_since_restore: 1269
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 4480.930234909058
time_this_iter_s: 3.9570114612579346
time_total_s: 4480.930234909058
timestamp

custom_metrics: {}
date: 2019-01-31_18-20-51
done: false
episode_len_mean: 280.7
episode_reward_max: 769.6073472960825
episode_reward_mean: 708.6297849718342
episode_reward_min: 329.1185875179815
episodes_this_iter: 14
episodes_total: 19294
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.7306471467018127
    cur_lr: 4.999999873689376e-05
    entropy: -0.09025371074676514
    kl: 0.01264545414596796
    policy_loss: -0.005331998225301504
    total_loss: 421.0212097167969
    vf_explained_var: 0.9056607484817505
    vf_loss: 421.01727294921875
  grad_time_ms: 1160.785
  load_time_ms: 1.587
  num_steps_sampled: 5108000
  num_steps_trained: 5108000
  sample_time_ms: 2390.406
  update_time_ms: 7.835
iterations_since_restore: 1277
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 4509.09996342659
time_this_iter_s: 3.4585914611816406
time_total_s: 4509.09996342659
timestamp: 15

custom_metrics: {}
date: 2019-01-31_18-21-19
done: false
episode_len_mean: 230.45
episode_reward_max: 766.7464863261742
episode_reward_mean: 568.455366165397
episode_reward_min: 62.89306823230824
episodes_this_iter: 15
episodes_total: 19428
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.36532357335090637
    cur_lr: 4.999999873689376e-05
    entropy: 0.045274969190359116
    kl: 0.012122525833547115
    policy_loss: -0.016821179538965225
    total_loss: 586.2947387695312
    vf_explained_var: 0.8617404103279114
    vf_loss: 586.3070678710938
  grad_time_ms: 1126.91
  load_time_ms: 1.589
  num_steps_sampled: 5140000
  num_steps_trained: 5140000
  sample_time_ms: 2327.789
  update_time_ms: 7.787
iterations_since_restore: 1285
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 4536.942364454269
time_this_iter_s: 3.5127766132354736
time_total_s: 4536.942364454269
timestamp: 

custom_metrics: {}
date: 2019-01-31_18-21-47
done: false
episode_len_mean: 267.9
episode_reward_max: 770.0422049655668
episode_reward_mean: 674.3265459205145
episode_reward_min: 40.043488621447075
episodes_this_iter: 14
episodes_total: 19545
experiment_id: 3d9aec997d3a4c69a7a0e25b9cdf8784
hostname: ip-172-31-4-39
info:
  default:
    cur_kl_coeff: 0.5479854941368103
    cur_lr: 4.999999873689376e-05
    entropy: -0.02797182835638523
    kl: 0.005165833979845047
    policy_loss: -0.0046679298393428326
    total_loss: 409.3791198730469
    vf_explained_var: 0.9147530198097229
    vf_loss: 409.38092041015625
  grad_time_ms: 1123.49
  load_time_ms: 1.609
  num_steps_sampled: 5172000
  num_steps_trained: 5172000
  sample_time_ms: 2305.876
  update_time_ms: 7.52
iterations_since_restore: 1293
node_ip: 172.31.4.39
num_metric_batches_dropped: 0
pid: 11276
policy_reward_mean: {}
time_since_restore: 4564.465911865234
time_this_iter_s: 3.4550204277038574
time_total_s: 4564.465911865234
timestamp:

In [31]:
checkpoint_path = agent.save()
#checkpoint_path = "/home/ubuntu/ray_results/PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_2019-01-30_16-56-073zc25tzh/checkpoint_110/checkpoint-110"
print(checkpoint_path)


/home/ubuntu/ray_results/PPO_JSBSim-HeadingControlTask-A320-NoFG-v0_2019-01-31_17-01-25ktvfzdev/checkpoint_1300/checkpoint-1300


# build json output function

In [26]:
import configparser
config_ini = configparser.ConfigParser()
config_ini.read('config-state-action.ini')

### collect state var from config file
state_list = config_ini.get('SA_DEFAULT', 'states').split('\n')
action_list = config_ini.get('SA_DEFAULT', 'actions').split('\n')
print(state_list)
print(action_list)

def add_state_action_to_dict(sa_dict, sa):
    """
    add last state / action values to the dictionnary
    from cong ini file
    """
    if sa_dict == None:
        sa_dict = {}
        for s in state_list:
            sa_dict[s]=[]
        for a in action_list:
            sa_dict[a]=[]
    else:
        i = 0
        for s in state_list:
            sa_dict[s].append(str(sa[i]))
            i += 1
        for a in action_list:
            sa_dict[a].append(str(sa[i]))
            i += 1
            
    return sa_dict
                                               
def send_to_sqs(sa_dict):
    """
    send the dictionnary to the AWS queue: L2F.fifo (https://sqs.eu-west-1.amazonaws.com/190334810943/L2F.fifo)
    """
    
    # Get the service resource
    sqs = boto3.resource('sqs', region_name='eu-west-1')

    # Get the queue
    queue = sqs.get_queue_by_name(QueueName='L2F.fifo')
    
    #with open('result.json', 'w') as fp:
    #    json.dump(sa_dict, fp)

    # send json file
    response = queue.send_message(
        MessageBody=sa_dict,
        MessageGroupId='state_action_history'
    )

    # The response is NOT a resource, but gives you a message ID and MD5
    print(response.get('MessageId'))
    print(response.get('MD5OfMessageBody'))

['altitude_sl_ft', 'heading_deg', 'pitch_rad', 'roll_rad', 'v_north_fps', 'v_east_fps', 'v_down_fps']
['aileron_cmd', 'elevator_cmd', 'rudder_cmd', 'throttle_cmd']


# let's test our model

In [27]:
trained_config = config.copy()
test_agent = PPOAgent(trained_config, 'JSBSim-HeadingControlTask-A320-NoFG-v0')
test_agent.restore(checkpoint_path)

2019-01-31 17:20:13,216	WARNING ppo.py:137 -- By default, observations will be normalized with MeanStdFilter
2019-01-31 17:20:13,219	INFO policy_evaluator.py:260 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)


state_variables =  (BoundedProperty(name='position/h-sl-ft', description='altitude above mean sea level [ft]', min=-1400, max=85000), BoundedProperty(name='attitude/psi-deg', description='heading [deg]', min=0, max=360), BoundedProperty(name='attitude/pitch-rad', description='pitch [rad]', min=-1.5707963267948966, max=1.5707963267948966), BoundedProperty(name='attitude/roll-rad', description='roll [rad]', min=-3.141592653589793, max=3.141592653589793), BoundedProperty(name='velocities/v-north-fps', description='velocity true north [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-east-fps', description='velocity east [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-down-fps', description='velocity downwards [ft/s]', min=-inf, max=inf))
action_variables =  (BoundedProperty(name='fcs/aileron-cmd-norm', description='aileron commanded position, normalised', min=-1.0, max=1.0), BoundedProperty(name='fcs/elevator-cmd-norm', description='elevator commanded positio

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-01-31 17:20:14,277	INFO multi_gpu_optimizer.py:74 -- LocalMultiGPUOptimizer devices ['/gpu:0']


In [29]:
env = gym.make('JSBSim-HeadingControlTask-A320-NoFG-v0')
obs_space = env.observation_space
act_space = env.action_space
print(obs_space, act_space)
state = env.reset()
done = False
cumulative_reward = 0
i = 0
act_stat_reward = []
sa_dict = None
while i != 1000:
    #print(gym.tasks.last_state())
    action = test_agent.compute_action(state)
    state, reward, done, _ = env.step(action)
    print(action, state, reward)
    mergedlist = []
    mergedlist.extend(state)
    mergedlist.extend(action)
    state_action_list = mergedlist
    sa_dict = add_state_action_to_dict(sa_dict, state_action_list)
    
    mergedlist.extend([reward])
    
    act_stat_reward.append(mergedlist)
    cumulative_reward += reward
    i += 1

print("SA_DICT", sa_dict)

### Send the state action json file to the AWS queue
send_to_sqs(json.dumps(sa_dict))

print(cumulative_reward)
print(",".join(str(v) for v in state_list+action_list))
print("aileron_cmd,elevator_cmd,rudder_cmd,pitch_rad,roll_rad,sideslip_deg,v_north_fps,v_east_fps,altitude_sl_ft,v_down_fps,p_radps,q_radps,r_radps,reward")
for e in act_stat_reward:
        print(",".join(str(v) for v in e))
with open('result.json', 'w') as fp:
    json.dump(sa_dict, fp)

state_variables =  (BoundedProperty(name='position/h-sl-ft', description='altitude above mean sea level [ft]', min=-1400, max=85000), BoundedProperty(name='attitude/psi-deg', description='heading [deg]', min=0, max=360), BoundedProperty(name='attitude/pitch-rad', description='pitch [rad]', min=-1.5707963267948966, max=1.5707963267948966), BoundedProperty(name='attitude/roll-rad', description='roll [rad]', min=-3.141592653589793, max=3.141592653589793), BoundedProperty(name='velocities/v-north-fps', description='velocity true north [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-east-fps', description='velocity east [ft/s]', min=-inf, max=inf), BoundedProperty(name='velocities/v-down-fps', description='velocity downwards [ft/s]', min=-inf, max=inf))
action_variables =  (BoundedProperty(name='fcs/aileron-cmd-norm', description='aileron commanded position, normalised', min=-1.0, max=1.0), BoundedProperty(name='fcs/elevator-cmd-norm', description='elevator commanded positio

[ 0.21687175 -0.30063415 -0.34057644  0.4688048 ] [ 5.91510308e+03  1.76429795e+02  4.96526014e-02  1.05273971e+00
 -6.31592883e+02  8.88893568e+01 -2.77632194e+01] 2.476480210897961
[ 0.12953588 -0.260638   -0.23349053  0.461185  ] [ 5.92056452e+03  1.76084382e+02  3.59408114e-02  1.11239440e+00
 -6.32389084e+02  7.68638141e+01 -2.64930555e+01] 2.4906913842449443
[-0.01248265 -0.18114482 -0.11880383  0.4069325 ] [ 5.92558835e+03  1.75518748e+02  2.92765934e-02  1.15992966e+00
 -6.33093288e+02  6.74455716e+01 -2.35260788e+01] 2.503075259909062
[-0.18714571 -0.15411219  0.01818176  0.32660282] [ 5.92991894e+03  1.75077369e+02  3.14768982e-02  1.17804035e+00
 -6.33690837e+02  6.02705429e+01 -1.97238714e+01] 2.513878639316655
[-0.22723447 -0.16624206  0.01662147  0.42149812] [ 5.93348429e+03  1.75182631e+02  3.78660784e-02  1.15878558e+00
 -6.34182786e+02  5.43362641e+01 -1.60079273e+01] 2.5240747757356323
[-0.22186244 -0.1757271   0.04296277  0.37133676] [ 5.93637233e+03  1.75993114e+02 

[-0.3927284  -0.33893806  1.1411986  -0.1031653 ] [ 5.96419104e+03  1.99389347e+02  8.42789325e-02  6.51259535e-01
 -5.46616848e+02 -2.63462212e+02  2.14199457e+01] 2.676612197703493
[ 0.22431691 -0.19377032  0.6191293  -0.41827378] [ 5.96054799e+03  2.02169469e+02  1.44071070e-01  5.52717863e-01
 -5.44269430e+02 -2.66216605e+02  1.35696281e+01] 2.6742151694395986
[ 0.12606558  0.04727409  0.30589637 -0.04635556] [ 5.95915178e+03  2.04712560e+02  1.57617306e-01  4.91433118e-01
 -5.39878670e+02 -2.71379995e+02  1.91657748e-01] 2.6774009583395015
[0.54448104 0.00884968 0.3963382  0.2516387 ] [ 5.96033555e+03  2.05610685e+02  1.25467330e-01  4.67096775e-01
 -5.35759528e+02 -2.75998911e+02 -1.11471714e+01] 2.6831839852119974
[1.1788652  0.2985258  0.21087673 0.0631257 ] [ 5.96319529e+03  2.04864971e+02  6.34540721e-02  5.14177888e-01
 -5.33221203e+02 -2.78715033e+02 -1.61263304e+01] 2.68905626938584
[0.7430743  0.3039387  0.19292974 0.4827249 ] [ 5.96620768e+03  2.03189333e+02 -1.34883845e

[ 2.508668  -0.6977469 -2.2982826 -1.0517578] [ 5.94210649e+03  2.21334516e+02  1.91699492e-01  6.93845278e-01
 -4.32039296e+02 -3.35099247e+02  1.58700272e+01] 2.6762250073921736
[ 0.1578487  -0.06174503  2.2353382   0.48467028] [ 5.94018922e+03  2.26283544e+02  2.18724252e-01  8.04644600e-01
 -4.21232988e+02 -3.44953285e+02  3.33174110e+00] 2.684646552994735
[ 0.3464501  -0.42765957 -0.9545711   1.6198226 ] [ 5.94063621e+03  2.27690773e+02  1.96040043e-01  8.78880646e-01
 -4.09270770e+02 -3.54856348e+02 -7.25421901e+00] 2.6897971707374726
[ 0.17343292 -0.4232118   0.87036586  0.9702898 ] [ 5.94280475e+03  2.27570594e+02  1.35145874e-01  9.38614107e-01
 -3.98105123e+02 -3.64196754e+02 -1.37076010e+01] 2.69548707575333
[-0.21926978  2.0508053   2.1553068   0.7990788 ] [ 5.94595615e+03  2.22331357e+02  1.03778711e-01  9.60977021e-01
 -3.89433358e+02 -3.71333650e+02 -1.68262928e+01] 2.6896875108450056
[-1.2276076   1.7070372   0.06702101  0.63011974] [ 5.94908854e+03  2.14287884e+02  2.7

[0.18683302 0.17686906 4.1029925  0.8895128 ] [ 5.90653781e+03  2.19689180e+02  1.43476675e-01  9.97844970e-01
 -3.65195384e+02 -3.17672329e+02  1.97361439e+01] 2.5728527418611775
[-0.6596843  1.279168   3.6185946  1.2530072] [ 5.90283278e+03  2.17976954e+02  1.20298238e-01  9.19662438e-01
 -3.60530200e+02 -3.22075280e+02  1.74993027e+01] 2.557804818297753
[-1.4057704 -2.6693401  6.735538   1.2683651] [ 5.89923151e+03  2.15089533e+02  8.19028917e-02  7.74628189e-01
 -3.59895928e+02 -3.22292796e+02  1.88941052e+01] 2.539780669688878
[-0.82989955  1.1096423   6.900324    0.41819042] [ 5.89541608e+03  2.12874760e+02  4.14507563e-02  5.75790468e-01
 -3.59469290e+02 -3.22270013e+02  1.93418526e+01] 2.5230301207060544
[4.5872784  0.69389725 6.220338   2.0646207 ] [ 5.89128563e+03  2.10099013e+02 -3.89917238e-02  4.09295756e-01
 -3.60939470e+02 -3.20186002e+02  2.26394336e+01] 2.5038462238705375
[-0.8176507 -0.9212235 -5.360491  -1.8840133] [ 5.88590581e+03  2.10158138e+02 -1.21329776e-01  2.

[  4.4909487  -0.8610363 -14.829743   -4.582017 ] [ 5.76787494e+03  2.22398618e+02  1.84540821e-01 -2.16602496e-01
 -3.15120962e+02 -3.20363208e+02  1.08569479e+01] 2.1952043172323266
[ 3.2558455 -0.3063391 -8.462249  -0.776966 ] [ 5.76652643e+03  2.24558590e+02  1.87956539e-01 -2.66498503e-01
 -3.16976249e+02 -3.17492610e+02  2.76199354e+00] 2.197458396381999
[1.8367424 1.5008833 3.2228353 1.5978577] [ 5.76681483e+03  2.26913173e+02  1.35911732e-01 -2.33541943e-01
 -3.18086853e+02 -3.14996128e+02 -5.14834237e+00] 2.2047999965198963
[ 3.8812015 -0.6767357 23.41001    0.5027206] [ 5.76797436e+03  2.25653264e+02  2.79626205e-02 -1.50357288e-01
 -3.18398927e+02 -3.14666106e+02 -5.20158823e+00] 2.2045211745050444
[-8.032374   -2.0154078  14.222493   -0.02524127] [ 5.76849942e+03  2.21781815e+02 -5.42551629e-02 -8.99870979e-02
 -3.18943617e+02 -3.14663947e+02  6.48782304e-01] 2.195225661830018
[-11.982549   -2.901613  -15.586906   -1.3995101] [ 5.76756320e+03  2.19008589e+02 -8.27490931e-02

[ -7.080045  -4.388215 -20.684708  26.347965] [ 863.11195355  202.30510265   -0.93686108   -1.03395164 -282.02911988
 -179.29458779  561.96912612] -11.484952621671509
[ 1.7083836   0.43017912 13.91436    14.640299  ] [ 750.84167309  206.35140708   -0.96889153   -1.15205778 -300.78803024
 -165.17218215  561.4754359 ] -11.785574777307666
[-23.683619    -2.8657727   21.79207     -0.41623175] [ 638.07135199  208.26890647   -1.08461782   -1.21099756 -304.17784855
 -161.66856306  567.01921642] -12.09349928205728
[ 9.464952    0.58019686 28.087008   -7.901521  ] [ 523.95811453  206.35303369   -1.1559264    -1.25138971 -305.59527549
 -154.67733449  574.33449912] -12.415802366035873
[ 52.98724    -2.0763123 -19.23997    -5.59867  ] [ 408.2004963   206.53274556   -1.12455273   -1.17956801 -301.33345297
 -152.14426342  583.10847208] -12.736852105928769
[26.689417   0.8253324  5.9111986  4.5298057] [ 291.07445998  203.62533178   -1.05774166   -0.97326334 -308.39762775
 -144.50866023  587.53074254]

[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan

[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan

[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
[nan nan nan nan] [nan nan nan nan nan nan nan] nan
SA_DICT {'altitude_sl_ft': ['5002.845688454807', '5011.9077603928745', '5028.055201318115', '5049.130153011531', '5070.911927528679', '5089.509031493217', '5104.8273602